In [ ]:
pip install opencv-python matplotlib numpy

  Using cached opencv_python-4.13.0.90-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-macosx_11_0_arm64.whl.metadata (52 kB)
  Using cached contourpy-1.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (5.5 kB)


In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def sort_contours_grid(contours, error_margin=20):
    """
    Sorts contours from top-to-bottom, then left-to-right.
    Crucial for maintaining the correct order of tiles.
    """
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    c_boxes = list(zip(contours, bounding_boxes))
    
    # Sort primarily by Y (top to bottom)
    c_boxes.sort(key=lambda b: b[1][1])
    
    rows = []
    current_row = []
    last_y = -1
    
    for c, box in c_boxes:
        x, y, w, h = box
        # Group by Y proximity
        if last_y == -1 or abs(y - last_y) < (h // 2): 
            current_row.append((c, box))
        else:
            current_row.sort(key=lambda b: b[1][0]) # Sort row by X
            rows.extend(current_row)
            current_row = [(c, box)]
        last_y = y

    if current_row:
        current_row.sort(key=lambda b: b[1][0])
        rows.extend(current_row)
        
    return [pair[0] for pair in rows]

def display_in_notebook(images, titles=None):
    """
    Helper function to display a list of images in a grid within Jupyter.
    """
    n = len(images)
    if n == 0: return
    
    cols = 4
    rows = (n + cols - 1) // cols
    
    plt.figure(figsize=(15, 3 * rows))
    
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i + 1)
        # Convert BGR (OpenCV default) to RGB (Matplotlib default)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb_img)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def extract_tiles(image_path, output_dir="extracted_tiles", show_plot=True):
    """
    Main function to detect, sort, save, and display tiles.
    """
    if not os.path.exists(image_path):
        print(f"Error: File '{image_path}' not found.")
        return

    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Could not read image. Please try uploading again.")
        return
    
    # Preprocessing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 30, 150)
    dilated = cv2.dilate(edged, None, iterations=1)

    # Find Contours
    contours, _ = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter Contours
    tile_contours = []
    image_area = img.shape[0] * img.shape[1]
    
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = w * h
        aspect_ratio = w / float(h)
        
        # Filter by area (1% to 90% of image) and aspect ratio (approx square)
        if area > (image_area * 0.01) and area < (image_area * 0.9):
            if 0.5 < aspect_ratio < 2.0:
                tile_contours.append(c)

    if not tile_contours:
        print("No tiles detected.")
        return

    sorted_contours = sort_contours_grid(tile_contours)

    # Extract, Save, and Collect for Display
    extracted_images = []
    titles = []
    margin = 5
    
    print(f"Found {len(sorted_contours)} tiles. Saving to '{output_dir}/'...")

    for i, c in enumerate(sorted_contours):
        x, y, w, h = cv2.boundingRect(c)
        
        # Crop with margin
        x_start = max(0, x + margin)
        y_start = max(0, y + margin)
        x_end = min(img.shape[1], x + w - margin)
        y_end = min(img.shape[0], y + h - margin)

        roi = img[y_start:y_end, x_start:x_end]
        extracted_images.append(roi)
        titles.append(f"Tile {i+1}")
        
        # Save to disk
        filename = output_path / f"tile_{i+1:02d}.jpg"
        cv2.imwrite(str(filename), roi)

    # Display in Notebook
    if show_plot:
        print("Displaying extracted tiles...")
        display_in_notebook(extracted_images, titles)

def create_uploader():
    """
    Creates a Jupyter widget to upload files and triggers processing immediately.
    """
    uploader = widgets.FileUpload(
        accept='image/*',  # Accept all image types
        multiple=False
    )
    
    output = widgets.Output()
    
    def on_upload_change(change):
        if not change['new']: return
        
        with output:
            clear_output()
            # ipywidgets 8+ returns a tuple of dicts
            # We take the most recent upload
            uploaded_file = change['new'][0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
            
            # Save the uploaded file to disk locally so OpenCV can read it
            with open(filename, 'wb') as f:
                f.write(content)
                
            print(f"File '{filename}' uploaded successfully.")
            extract_tiles(filename)

    uploader.observe(on_upload_change, names='value')
    
    display(widgets.VBox([
        widgets.Label("Upload an image to extract tiles:"),
        uploader,
        output
    ]))

# Run the uploader interface
if __name__ == "__main__":
    create_uploader()